In [ ]:
# This is a toy function that we wish to analyze in Phylanx
def fib(n):
    if n < 2:
        return n
    else:
        return fib(n-1)+fib(n-2)

In [ ]:
from run_local import run_local
job = run_local(fib,(10,),localities=1)

In [ ]:
job.wait()
job.viz()

In [ ]:
def cannon(size):
    array1 = random_d([size, size], find_here(), num_localities())
    array2 = random_d([size, size], find_here(), num_localities())
    cannon_product_d(array1, array2)

In [ ]:
# Start the job locally. Not that canon needs 4 localties to run correctly.
job = run_local(cannon,(120,),localities=4)

# Wait for the job to finish
job.wait()

In [ ]:
# Visualize the job in traveler
job.viz()

In [ ]:
# Download all the perf data from the job as a tarball
job.download()

In [ ]:
# In this cell, we run the function locally and collect performance data
from visualizeInTraveler import visualizeInTraveler
from phylanx import Phylanx

# If OTF2 data is already present, we need to delete it or
# move it out of the way before we run our function
!rm -fr OTF2_archive

# Generate a Phylanx version of the function, tell it to
# collect performance data
pfib = Phylanx(fib,performance='x')

# Run the function
pfib(11)

# Visualize the performance data
visualizeInTraveler(pfib,verbose=True)

In [ ]:
# If we wish to run remotely using Tapis/Agave, we
# need to load the JetLag interface.
from jetlag_setup import uv

In [ ]:
from remote_run import remote_run, to_string, viz
from jetlag import Universal, RemoteJobWatcher
from knownsystems import *
import sys
from random import randint
from gui_fun import gui_fun, settings

In [ ]:
# Create a random argument for our fib function
fibno = randint(10,15)
print('fib(',fibno,')=...',sep='',flush=True)

# Submit the function to run remotely
job = remote_run(uv, fib, (fibno,), queue='queue', nodes=1, ppn=1)

# Optionally wait for the result
job.wait()

# If the result is ready, fetch it
print("result:",job.get_result())

# Visualize the result
viz(job, verbose=True)

In [ ]:
job.diag()

In [ ]:
# You can look at data from an old job if you supply the full job id
# job = RemoteJobWatcher(uv, "699cf0f8-0cb6-4f6a-a215-16c14c7522d4-007")
# print("result:",job.get_result())

In [ ]:
%%html
<iframe src="http://localhost:8000/static/interface.html" style='width: 100%; height: 300px;'></iframe>

In [ ]:
!curl -LO https://raw.githubusercontent.com/STEllAR-GROUP/phylanx/master/examples/algorithms/datasets/breast_cancer.csv

In [ ]:
import numpy as np

def lra_demo(x, y, alpha, iterations, enable_output):
    weights = np.zeros(np.shape(x)[1])
    transx = np.transpose(x)
    pred = np.zeros(np.shape(x)[0])
    error = np.zeros(np.shape(x)[0])
    gradientx = np.zeros(np.shape(x)[1])
    step = 0
    while step < iterations:
        if (enable_output):
            print("step: ", step, ", ", weights)
        pred = 1.0 / (1.0 + np.exp(-np.dot(x, weights)))
        error = pred - y
        gradientx = np.dot(transx, error)
        weights = weights - (alpha * gradientx)
        step += 1
    return weights

file_name = "breast_cancer.csv"

data = np.genfromtxt(file_name, skip_header=1, delimiter=",")
x = data[:, :-1]
y = data[:, -1:]
y = np.reshape(y, (y.shape[0], ))
#res = lra(x, y, 1e-5, 750, 0)

print("args:",(x, y, 1e-5, 750, 0))

job = remote_run(uv, lra_demo, (x, y, 1e-5, 750, 0), queue='fork', nodes=1, ppn=1)
job.wait()

In [ ]:
job.diag()

In [ ]:
print("result:",job.get_result())
viz(job)

In [ ]:
import numpy as np


def ALS(ratings, regularization, num_factors, iterations, alpha, enable_output):
    num_users = np.shape(ratings)[0]
    num_items = np.shape(ratings)[1]

    conf = alpha * ratings
    conf_u = np.zeros((num_items, 1))
    conf_i = np.zeros((num_items, 1))

    c_u = np.zeros((num_items, num_items))
    c_i = np.zeros((num_users, num_users))
    p_u = np.zeros((num_items, 1))
    p_i = np.zeros((num_users, 1))

    I_f = np.identity(num_factors)
    I_i = np.identity(num_items)
    I_u = np.identity(num_users)

    set_seed(0)  # noqa: F821
    X = random([num_users, num_factors])  # noqa: F821
    Y = random([num_items, num_factors])  # noqa: F821

    i = 0
    u = 0
    k = 0

    XtX = np.zeros((num_factors, num_factors))
    YtY = np.zeros((num_factors, num_factors))
    A = np.zeros([num_factors, num_factors])
    b = np.zeros([num_factors])
    while k < iterations:
        if enable_output:
            print("iteration ", k)
            print("X: ", X)
            print("Y: ", Y)
        YtY = np.dot(np.transpose(Y), Y) + regularization * I_f
        XtX = np.dot(np.transpose(X), X) + regularization * I_f
        while u < num_users:
            conf_u = conf[u, :]
            c_u = np.diag(conf_u)
            p_u = conf_u != 0
            A = YtY + np.dot(np.dot(np.transpose(Y), c_u), Y)
            b = np.dot(np.dot(np.transpose(Y), c_u + I_i), np.transpose(p_u))
            X[u, :] = np.dot(inverse(A), b)  # noqa: F821
            u = u + 1
        while i < num_items:
            conf_i = conf[:, i]
            c_i = np.diag(conf_i)
            p_i = conf_i != 0
            A = XtX + np.dot(np.dot(np.transpose(X), c_i), X)
            b = np.dot(np.dot(np.transpose(X), c_i + I_u), np.transpose(p_i))
            Y[i, :] = np.dot(inverse(A), b)  # noqa: F821
            i = i + 1
        u = 0
        i = 0
        k = k + 1
    return [X, Y]


# test example
ratings = np.zeros((10, 5))
ratings[0, 1] = 4
ratings[1, 0] = 1
ratings[1, 2] = 4
ratings[1, 4] = 5
ratings[2, 3] = 2
ratings[3, 1] = 8
ratings[4, 2] = 4
ratings[6, 4] = 2
ratings[7, 0] = 1
ratings[8, 3] = 5
ratings[9, 0] = 1
ratings[9, 3] = 2

regularization = 0.1
num_factors = 3
iterations = 5
alpha = 40
enable_output = False

#result = ALS(ratings, regularization, num_factors, iterations, alpha, enable_output)
#print(" X = ", result[0])
#print(" Y = ", result[1])
job = remote_run(uv, ALS, (ratings, regularization, num_factors, iterations, alpha, enable_output), queue='fork', nodes=1, ppn=1)
job.wait()

In [ ]:
job.diag()

In [ ]:
print(job.std_output())

In [ ]:
print(job.err_output())

In [ ]:
print(job.get_result())
viz(job)